In [1]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/node-master'       # path to your project on Drive

GIT_USERNAME = "sgsyang" # replace with yours
#GIT_TOKEN = "XXX"           # definitely replace with yours
#GIT_REPOSITORY = "yyy"      # ...nah


drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
GIT_PATH = "https://github.com/sgsyang/node.git"

!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/node-master’: File exists
Cloning into 'node'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 0), reused 19 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.
mv: cannot stat './temp/*': No such file or directory
sending incremental file list
README.md
          1,555 100%    0.00k

In [2]:

!pip install category_encoders
!pip install tensorboardX
# !pip install qhoptim

%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0,1
import os, sys
import time
sys.path.insert(0, '..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import lib
import torch, torch.nn as nn
import torch.nn.functional as F
# from qhoptim.pyt import QHAdam









device = 'cuda' if torch.cuda.is_available() else 'cpu'

experiment_name = 'Shelter_Catoost'
experiment_name = '{}_{}.{:0>2d}.{:0>2d}_{:0>2d}:{:0>2d}'.format(experiment_name, *time.gmtime()[:5])
print("experiment:", experiment_name)

     |████████████████████████████████| 81kB 3.4MB/s 
     |████████████████████████████████| 204kB 4.9MB/s 
env: CUDA_VISIBLE_DEVICES=0,1


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


experiment: Shelter_Catoost_2020.05.28_00:07


In [3]:
torch.cuda.get_device_name(device)

'Tesla P100-PCIE-16GB'

##Default Parameters

In [0]:
# newdata = lib.Dataset("SHELTER", random_state = 133, quantile_transform=True, quantile_noise=1e-3)
# print(newdata.X_train.shape)
# print(newdata.y_train.shape)

# print(newdata.X_test.shape)
# print(newdata.y_test.shape)



(17908, 5)
(17908,)
(11456, 5)
(11456,)


In [8]:
!pip install catboost
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score


random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]
# random_state = [1337]
def Test(random_state):
  data = lib.Dataset("SHELTER", random_state = random_state, quantile_transform=True, quantile_noise=1e-3)
  num_features = data.X_train.shape[1]
  num_classes = len(set(data.y_train))
  model = CatBoostClassifier(loss_function='MultiClass',task_type="GPU",
                           devices='0:1') 
  model.fit(data.X_train, data.y_train,verbose = False)
  pred = model.predict(data.X_test)
  accuracy = accuracy_score(data.y_test, pred)

  error = 1-accuracy
  print('Catboost classification for shelter')
  print('The Error:',error)
  print(f'Test {random_state} is Over--')
  return error,random_state


score_results = [] 
key_results = []

for i in random_state:
  score, random_state = Test(i)
  score_results.append(score)
  key_results.append(random_state)




Catboost classification for shelter
The Error: 0.4676152234636871
Test 1337 is Over--
Catboost classification for shelter
The Error: 0.4676152234636871
Test 133 is Over--
Catboost classification for shelter
The Error: 0.4676152234636871
Test 13 is Over--
Catboost classification for shelter
The Error: 0.4676152234636871
Test 1 is Over--
Catboost classification for shelter
The Error: 0.4676152234636871
Test 602 is Over--
Catboost classification for shelter
The Error: 0.4676152234636871
Test 295 is Over--
Catboost classification for shelter
The Error: 0.4676152234636871
Test 970 is Over--
Catboost classification for shelter
The Error: 0.4676152234636871
Test 2000 is Over--
Catboost classification for shelter
The Error: 0.4676152234636871
Test 3000 is Over--
Catboost classification for shelter
The Error: 0.4676152234636871
Test 10000 is Over--


In [9]:
import pprint
from statistics import mean 


Result= dict(zip(key_results,score_results))

pprint.pprint(Result, width=1)


mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)

{1: 0.4676152234636871,
 13: 0.4676152234636871,
 133: 0.4676152234636871,
 295: 0.4676152234636871,
 602: 0.4676152234636871,
 970: 0.4676152234636871,
 1337: 0.4676152234636871,
 2000: 0.4676152234636871,
 3000: 0.4676152234636871,
 10000: 0.4676152234636871}
Mean: 0.4676152234636871
Max: 0.4676152234636871
Min: 0.4676152234636871
Distance to max 0.0
Distance to min: 0.0


##Tuned Parameters



In [10]:
!pip install hyperopt
import xgboost as xgb
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


!pip install catboost
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score


# the space 
space={'learning_rate':hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
       'random_strength':hp.quniform("random_strength", 1,20,1),
       'one_hot_max_size':hp.quniform("one_hot_max_size", 0,25,1),
       'l2_leaf_reg': hp.loguniform('l2_leaf_reg',1 ,10),
       'bagging_temperature':hp.uniform('bagging_temperature', 0, 1),
       'leaf_estimation_iterations':hp.quniform("leaf_estimation_iterations", 1,10,1)
    }

In [11]:
def my_tuning(space):
    reg=CatBoostClassifier( learning_rate=space['learning_rate'],
                          random_strength = int(space['random_strength']),
                          one_hot_max_size = int(space['one_hot_max_size']),
                          l2_leaf_reg = space['l2_leaf_reg'],
                          bagging_temperature = space['bagging_temperature'],
                          leaf_estimation_iterations = int(space['leaf_estimation_iterations']),
                          loss_function='MultiClass',task_type="GPU",
                          devices='0:1')
        
    reg.fit(data.X_train, data.y_train,verbose =False)
    pred = reg.predict(data.X_test)

    accuracy = accuracy_score(data.y_test, pred)

    error = 1-accuracy
    # print('Catboost classification for AULDT')
    # print('The Error:',error)
    # print(f'Test {random_state} is Over--')

    #change the metric if you like
    return {'loss':error, 'status': STATUS_OK }

score_results = [] 
key_results = []

random_state = [1337,133,13, 1, 602, 295, 970, 2000, 3000, 10000]
for i in random_state:
  data = lib.Dataset("SHELTER", random_state=i, quantile_transform=True, quantile_noise=1e-3)
  print(f'The dataset with random state {i} is used')

  trials = Trials()
  best = fmin(fn=my_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
  # print('Best parameters',best)
  tuned_loss = trials.best_trial['result']['loss']
  print('Best Loss',tuned_loss)
  final = tuned_loss
  print('FINAL Error:',final)
  score_results.append(final)
  key_results.append(i)

The dataset with random state 1337 is used
100%|██████████| 50/50 [09:23<00:00, 11.28s/it, best loss: 0.3373777932960894]
Best Loss 0.3373777932960894
FINAL Error: 0.3373777932960894
The dataset with random state 133 is used
100%|██████████| 50/50 [09:09<00:00, 10.99s/it, best loss: 0.33231494413407825]
Best Loss 0.33231494413407825
FINAL Error: 0.33231494413407825
The dataset with random state 13 is used
100%|██████████| 50/50 [09:14<00:00, 11.08s/it, best loss: 0.34313896648044695]
Best Loss 0.34313896648044695
FINAL Error: 0.34313896648044695
The dataset with random state 1 is used
100%|██████████| 50/50 [09:25<00:00, 11.31s/it, best loss: 0.34148044692737434]
Best Loss 0.34148044692737434
FINAL Error: 0.34148044692737434
The dataset with random state 602 is used
100%|██████████| 50/50 [09:22<00:00, 11.25s/it, best loss: 0.33798882681564246]
Best Loss 0.33798882681564246
FINAL Error: 0.33798882681564246
The dataset with random state 295 is used
100%|██████████| 50/50 [09:09<00:00, 1

In [13]:
import pprint
Result= dict(zip(key_results,score_results))

print('----')
pprint.pprint(Result, width=1)

mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)

----
{1: 0.34148044692737434,
 13: 0.34313896648044695,
 133: 0.33231494413407825,
 295: 0.294518156424581,
 602: 0.33798882681564246,
 970: 0.3314420391061452,
 1337: 0.3373777932960894,
 2000: 0.33598114525139666,
 3000: 0.2975733240223464,
 10000: 0.29670041899441346}
Mean: 0.3248516061452513
Max: 0.34313896648044695
Min: 0.294518156424581
Distance to max 0.018287360335195624
Distance to min: -0.030333449720670314


In [0]:
# !pip install hyperopt
# import xgboost as xgb
# import hyperopt
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [0]:
# !pip install catboost
# import numpy as np
# from catboost import CatBoostClassifier
# from sklearn.metrics import accuracy_score


# random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]

# def Test(random_state):
#   data = lib.Dataset("SHELTER", random_state = random_state, quantile_transform=True, quantile_noise=1e-3)
#   num_features = data.X_train.shape[1]
#   num_classes = len(set(data.y_train))
#   model = CatBoostClassifier(objective = 'CrossEntropy',task_type="GPU",
#                            devices='0:1') 
#   model.fit(data.X_train, data.y_train)
#   pred = model.predict(data.X_test)
#   accuracy = accuracy_score(data.y_test, pred)

#   error = 1-accuracy
#   print('Catboost classification for CLICK')
#   print('The Error:',error)
#   print(f'Test {random_state} is Over--')
#   return error,random_state


# score_results = [] 
# key_results = []

# for i in random_state:
#   score, random_state = Test(i)
#   score_results.append(score)
#   key_results.append(random_state)




In [0]:
# import pprint
# from statistics import mean 


# Result= dict(zip(key_results,score_results))

# pprint.pprint(Result, width=1)


# mean = sum(score_results) / len(score_results)
# print('Mean:',mean)
# print('Max:',max(score_results))
# print('Min:',min(score_results))
# print('Distance to max',max(score_results)-mean)
# print('Distance to min:',min(score_results)-mean)

In [0]:
# import math
# print(math.exp(5))

148.4131591025766


In [0]:
# import math

# space={'eta': hp.loguniform('eta',math.exp(-7) ,1),
#        'max_depth': hp.quniform("max_depth", 2,10,2),
#        'subsample': hp.uniform('subsample', 0.5, 1),
#        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
#        'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.5,1),
#        'min_child_weight' : hp.loguniform('min_child_weight', math.exp(-16),math.exp(5)),
#        'alpha' : hp.choice('alpha', [0,hp.loguniform('a',math.exp(-16),math.exp(2))]),
#        'lambda': hp.choice('lambda',[ 0,hp.loguniform('l',math.exp(-16),math.exp(2))]),
#        'gamma': hp.choice ('gamma', [0,hp.loguniform('g',math.exp(-16),math.exp(2))])
        
        
#     }


# # Regression: 
# def hyperparameter_tuning(space):
#     reg=xgb.XGBRegressor( eta = space['eta'],max_depth = int(space['max_depth']), subsample=space['subsample'],
#                          gamma = space['gamma'],reg_alpha = space['alpha'],reg_lambda = space['lambda'],
#                         #  min_child_weight=space['min_child_weight'],
#                          colsample_bytree=space['colsample_bytree'],
#                          colsample_bylevel = space['colsample_bylevel'])
    
#     evaluation = [( data.X_train, data.y_train), ( data.X_valid, data.y_valid)]
    
#     reg.fit(data.X_train, data.y_train,
#             eval_set=evaluation, eval_metric="rmse",
#             early_stopping_rounds=10,verbose=False)

#     pred = reg.predict(data.X_valid)
#     mse= mean_squared_error(data.y_valid, pred)
#     print ("SCORE:", mse)
#     #change the metric if you like
#     return {'loss':mse, 'status': STATUS_OK }